In [68]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor

from sklearn import preprocessing

In [87]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
# df_train.head()
df_test = pd.read_csv("test.csv")
# df_test.head()

In [88]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [89]:
#Remove useless features from the set
UselessNames=['feat_002', 'feat_003', 'feat_004', 'feat_008', 'feat_009', 'feat_010', 'feat_011', 'feat_012', 'feat_013', 'feat_014', 'feat_015', 'feat_016', 'feat_017', 'feat_018', 'feat_019', 'feat_020', 'feat_021', 'feat_022', 'feat_023', 'feat_024', 'feat_026', 'feat_027', 'feat_028', 'feat_029', 'feat_030', 'feat_031', 'feat_032', 'feat_033', 'feat_034', 'feat_035', 'feat_036', 'feat_038', 'feat_039', 'feat_040', 'feat_041', 'feat_042', 'feat_043', 'feat_045', 'feat_046', 'feat_047', 'feat_048', 'feat_049', 'feat_050', 'feat_051', 'feat_052', 'feat_053', 'feat_054', 'feat_055', 'feat_056', 'feat_057', 'feat_058', 'feat_059', 'feat_060', 'feat_061', 'feat_062', 'feat_063', 'feat_064', 'feat_065', 'feat_066', 'feat_067', 'feat_070', 'feat_071', 'feat_073', 'feat_074', 'feat_075', 'feat_076', 'feat_077', 'feat_078', 'feat_079', 'feat_080', 'feat_081', 'feat_082', 'feat_083', 'feat_084', 'feat_085', 'feat_086', 'feat_088', 'feat_089', 'feat_091', 'feat_092', 'feat_093', 'feat_094', 'feat_095', 'feat_096', 'feat_097', 'feat_098', 'feat_099', 'feat_100', 'feat_101', 'feat_103', 'feat_104', 'feat_105', 'feat_106', 'feat_107', 'feat_108', 'feat_109', 'feat_110', 'feat_111', 'feat_112', 'feat_113', 'feat_114', 'feat_115', 'feat_116', 'feat_117', 'feat_118', 'feat_120', 'feat_121', 'feat_122', 'feat_124', 'feat_125', 'feat_127', 'feat_128', 'feat_129', 'feat_130', 'feat_131', 'feat_133', 'feat_134', 'feat_135', 'feat_136', 'feat_137', 'feat_138', 'feat_139', 'feat_140', 'feat_141', 'feat_142', 'feat_143', 'feat_144', 'feat_145', 'feat_146', 'feat_147', 'feat_148', 'feat_149', 'feat_150', 'feat_151', 'feat_152', 'feat_153', 'feat_154', 'feat_155', 'feat_156', 'feat_157', 'feat_158', 'feat_159', 'feat_160', 'feat_161', 'feat_162', 'feat_163', 'feat_164', 'feat_165', 'feat_166', 'feat_167', 'feat_168', 'feat_169', 'feat_170', 'feat_171', 'feat_172', 'feat_174', 'feat_175', 'feat_177', 'feat_178', 'feat_179', 'feat_180', 'feat_181', 'feat_182', 'feat_183', 'feat_184', 'feat_185', 'feat_186', 'feat_188', 'feat_189', 'feat_190', 'feat_191', 'feat_192', 'feat_193', 'feat_194', 'feat_195', 'feat_197', 'feat_198', 'feat_201', 'feat_202', 'feat_203', 'feat_204', 'feat_205', 'feat_206', 'feat_207', 'feat_209', 'feat_210', 'feat_211', 'feat_212', 'feat_213', 'feat_214', 'feat_215', 'feat_216', 'feat_217', 'feat_219', 'feat_220', 'feat_221', 'feat_222', 'feat_223', 'feat_224', 'feat_227', 'feat_228', 'feat_229', 'feat_230', 'feat_231', 'feat_232', 'feat_233', 'feat_234', 'feat_235', 'feat_236', 'feat_237', 'feat_238', 'feat_239', 'feat_240', 'feat_241', 'feat_242', 'feat_244', 'feat_245', 'feat_246', 'feat_247', 'feat_249', 'feat_250', 'feat_253', 'feat_254', 'feat_255', 'feat_256']

df_train_clean = df_train.drop(UselessNames, axis=1)
df_test_clean = df_test.drop(UselessNames, axis=1)

In [90]:
#Convert all 0 values to 2's
df_train_clean=df_train_clean.replace(0,2)
df_test_clean=df_test_clean.replace(0,2)

In [91]:
#Scaller: Trying to make all values centered
#disabled for now, as we are working on other tests

# df_train2=df_train.drop(['smiles'],axis=1).head()

# smiles=df_train['smiles']

# scaler = preprocessing.StandardScaler().fit(df_train2)
# df_train2=scaler.transform(df_train2) 
# df_train2=pd.DataFrame(data=df_train2[0:,0:])
# #               index=df_train2[1:,0],
# #               columns=df_train2[0,1:])
# df_train2.insert(0, 'smiles', smiles)

In [92]:
#Partitioning the data into training and test sets
df_test_clean = df_test_clean.drop(['smiles'], axis=1)
df_train_clean = df_train_clean.drop(['smiles'], axis=1)
vals_test = df_test_clean.values
vals_train = df_train_clean.values

Test_num = 5;
piecesize = int(len(df_train)/5);

X_train_train = vals_train[:(Test_num-1)*piecesize]
X_train_test = vals_train[(Test_num-1)*piecesize:]
Y_train_train =  Y_train[:(Test_num-1)*piecesize]
Y_train_test = Y_train[(Test_num-1)*piecesize:]

X_train_train = preprocessing.scale(X_train_train.T)
X_train_train = X_train_train.T
X_train_test = preprocessing.scale(X_train_test.T)
X_train_test = X_train_test.T

print ("X Train Data Shape:", X_train_train.shape)
print ("X Test Data Shape:", X_train_test.shape)
print ("Y Train Data Shape:", Y_train_train.shape)
print ("Y Test Data Shape:", Y_train_test.shape)

('X Train Data Shape:', (800000, 31))
('X Test Data Shape:', (200000, 31))
('Y Train Data Shape:', (800000,))
('Y Test Data Shape:', (200000,))


In [93]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [105]:
# Linear Regression 
LR = LinearRegression(n_jobs=-1)
BT=time.time()
LR.fit(X_train_train, Y_train_train)
ET=time.time()
LR_pred = LR.predict(X_train_test)

LR_Err=RMSE(Y_train_test,LR_pred)

print ET-BT
print LR_Err

1.0204179287
0.299252152197


In [106]:
# Random Forest Regression
RF = RandomForestRegressor(n_jobs=-1)
BT=time.time()
RF.fit(X_train_train, Y_train_train)
ET=time.time()
RF_pred = RF.predict(X_train_test)

RF_Err=RMSE(Y_train_test,RF_pred)
print ET-BT

print RF_Err


39.5637931824
0.273390285215


In [112]:
# Elastic Net Regression
EN=ElasticNetCV(n_jobs=-1)
BT=time.time()
EN.fit(X_train_train, Y_train_train)
ET=time.time()

EN_pred = EN.predict(X_train_test)

EN_Err=RMSE(Y_train_test,EN_pred)
       
print ET-BT
print EN_Err

103.575790882
0.299267569507


In [113]:
# Elastic Net Regression
MN=ElasticNet(alpha=0.8)
BT=time.time()
MN.fit(X_train_train, Y_train_train)
ET=time.time()

MN_pred = MN.predict(X_train_test)

MN_Err=RMSE(Y_train_test,MN_pred)
print ET-BT
print MN_Err

0.592020988464
0.407498540731


In [65]:
def basis_func_extension(X,d=10):
    curr_train = np.ones(X.shape)
    for i in np.arange(0, d):
        curr_train = np.vstack([curr_train.T,np.sin(X*1./(i+1)).T]).T
    return curr_train

FT=preprocessing.FunctionTransformer(basis_func_extension(X_train))
print dir(FT)
BR=time.time()
FT.fit(X_train,Y_train)
ET=time.time()

# FT_pred = FT.predict(X_test)


print ET-BT

['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__getstate__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_param_names', '_transform', 'accept_sparse', 'fit', 'fit_transform', 'func', 'get_params', 'inv_kw_args', 'inverse_func', 'inverse_transform', 'kw_args', 'pass_y', 'set_params', 'transform', 'validate']
19315.6042521


In [54]:
print 'EN Score =',EN.score(X_train,Y_train)
print 'RF Score =',RF.score(X_train,Y_train)
print 'LR Score =',LR.score(X_train,Y_train)
print 'MN Score =',MN.score(X_train,Y_train)

EN Score = 0.463186214763
RF Score = 0.641257258918
LR Score = 0.461816178672
MN Score = 0.00749168031858


In [149]:
EN

ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
       l1_ratio=0.5, max_iter=1000, n_alphas=100, n_jobs=1,
       normalize=False, positive=False, precompute='auto',
       random_state=None, selection='cyclic', tol=0.0001, verbose=0)

In [14]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [15]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)

In [97]:
def RMSE(Y,Y_Pred):
    N=len(Y)
    Err=0
    for i in xrange(N):
        Err+=(Y[i]-Y_Pred[i])**2
    Err=(1./N*Err)**(1./2.)
    return Err

In [140]:
names2=['feat_002', 'feat_003', 'feat_004', 'feat_008', 'feat_009', 'feat_010', 'feat_011', 'feat_012', 'feat_013', 'feat_014', 'feat_015', 'feat_016', 'feat_017', 'feat_018', 'feat_019', 'feat_020', 'feat_021', 'feat_022', 'feat_023', 'feat_024', 'feat_026', 'feat_027', 'feat_028', 'feat_029', 'feat_030', 'feat_031', 'feat_032', 'feat_033', 'feat_034', 'feat_035', 'feat_036', 'feat_038', 'feat_039', 'feat_040', 'feat_041', 'feat_042', 'feat_043', 'feat_045', 'feat_046', 'feat_047', 'feat_048', 'feat_049', 'feat_050', 'feat_051', 'feat_052', 'feat_053', 'feat_054', 'feat_055', 'feat_056', 'feat_057', 'feat_058', 'feat_059', 'feat_060', 'feat_061', 'feat_062', 'feat_063', 'feat_064', 'feat_065', 'feat_066', 'feat_067', 'feat_070', 'feat_071', 'feat_073', 'feat_074', 'feat_075', 'feat_076', 'feat_077', 'feat_078', 'feat_079', 'feat_080', 'feat_081', 'feat_082', 'feat_083', 'feat_084', 'feat_085', 'feat_086', 'feat_088', 'feat_089', 'feat_091', 'feat_092', 'feat_093', 'feat_094', 'feat_095', 'feat_096', 'feat_097', 'feat_098', 'feat_099', 'feat_100', 'feat_101', 'feat_103', 'feat_104', 'feat_105', 'feat_106', 'feat_107', 'feat_108', 'feat_109', 'feat_110', 'feat_111', 'feat_112', 'feat_113', 'feat_114', 'feat_115', 'feat_116', 'feat_117', 'feat_118', 'feat_120', 'feat_121', 'feat_122', 'feat_124', 'feat_125', 'feat_127', 'feat_128', 'feat_129', 'feat_130', 'feat_131', 'feat_133', 'feat_134', 'feat_135', 'feat_136', 'feat_137', 'feat_138', 'feat_139', 'feat_140', 'feat_141', 'feat_142', 'feat_143', 'feat_144', 'feat_145', 'feat_146', 'feat_147', 'feat_148', 'feat_149', 'feat_150', 'feat_151', 'feat_152', 'feat_153', 'feat_154', 'feat_155', 'feat_156', 'feat_157', 'feat_158', 'feat_159', 'feat_160', 'feat_161', 'feat_162', 'feat_163', 'feat_164', 'feat_165', 'feat_166', 'feat_167', 'feat_168', 'feat_169', 'feat_170', 'feat_171', 'feat_172', 'feat_174', 'feat_175', 'feat_177', 'feat_178', 'feat_179', 'feat_180', 'feat_181', 'feat_182', 'feat_183', 'feat_184', 'feat_185', 'feat_186', 'feat_188', 'feat_189', 'feat_190', 'feat_191', 'feat_192', 'feat_193', 'feat_194', 'feat_195', 'feat_197', 'feat_198', 'feat_201', 'feat_202', 'feat_203', 'feat_204', 'feat_205', 'feat_206', 'feat_207', 'feat_209', 'feat_210', 'feat_211', 'feat_212', 'feat_213', 'feat_214', 'feat_215', 'feat_216', 'feat_217', 'feat_219', 'feat_220', 'feat_221', 'feat_222', 'feat_223', 'feat_224', 'feat_227', 'feat_228', 'feat_229', 'feat_230', 'feat_231', 'feat_232', 'feat_233', 'feat_234', 'feat_235', 'feat_236', 'feat_237', 'feat_238', 'feat_239', 'feat_240', 'feat_241', 'feat_242', 'feat_244', 'feat_245', 'feat_246', 'feat_247', 'feat_249', 'feat_250', 'feat_253', 'feat_254', 'feat_255', 'feat_256']

In [147]:
df_curr = df_all.drop(names2, axis=1)


In [153]:
df_all.head()

,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,feat_010,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
def basis_func_extension(X,d=10):
    curr_train = np.ones(X.shape)
    for i in np.arange(0, d):
        curr_train = np.vstack([curr_train.T,np.sin(X*1./(i+1)).T]).T
    return curr_train

FT=preprocessing.FunctionTransformer(basis_func_extension(X_train))
